In [1]:
import torch
import pickle
torch.cuda.empty_cache()

# !pip install git+https://github.com/huggingface/transformers.git
# !pip install accelerate bitsandbytes xformers

from huggingface_hub import notebook_login
from tqdm import tqdm
from datetime import datetime

# notebook_login()
for i in range(torch.cuda.device_count()):
   print(i, "-->", torch.cuda.get_device_properties(i).name)

from torch import bfloat16
from transformers import pipeline

0 --> NVIDIA RTX A5000


2024-04-24 11:31:17.409575: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-24 11:31:17.433183: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 11:31:17.433203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 11:31:17.433812: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-24 11:31:17.437626: I tensorflow/core/platform/cpu_feature_guar

In [2]:
pipe = pipeline(
    "text-generation", 
    # model="HuggingFaceH4/zephyr-7b-beta", 
    model="meta-llama/Llama-2-7b-chat-hf",
    # model="meta-llama/Llama-2-7b-hf",
    # model="meta-llama/Llama-2-13b-chat-hf",
    # model="mistralai/Mistral-7B-Instruct-v0.1",
    torch_dtype=bfloat16, 
    # device_map="auto"
    # batch_size=1,
    device=0
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
data = []
with open("/local/sps-local/ner-role-extraction/relevant_sentences.pkl", "rb") as f:
    data = pickle.load(f)

print(f"{len(data)} sentences found")
data[2]

115537 sentences found


('/local/sps-local/docs/a5/57/6e/4e/a5576e4e89fc1ad271ba00b3694fde1d63a95e77d5f85102410dbbd65e95c05d.json',
 1,
 1185,
 [(1, 31, 48), (1, 75, 91), (3, 148, 154)],
 'Based on our experience in the β- and γ-C(sp3)-H\nfunctionalization of free carboxylic acids, we thus expected that\nthe identification of a suitable ligand would be crucial for the\ndevelopment of the desired alkynylation process.\n\n')

In [4]:
SYS_PROMPT = ('Do you agree with the provided question? Please answer with one word, either "yes" or "no".')
# SYS_PROMPT = ('Do you agree with the provided question? Please answer with "yes" or "no".')
# SYS_PROMPT = ('Do you agree with the question? Please answer using one word.')
# SYS_PROMPT = ('Your job is to answer my questions.')
# SYS_PROMPT = ('Your job is to answer my questions simply using yes or no.')
# SYS_PROMPT = ('')

def answer(sentence, chemical, role):
    # input = f'According to the sentence "{sentence}": Is {chemical} explicitly described as a {role}?'
    input = f'In the sentence "{sentence}": Is {chemical} explicitly described as {role}?'
    
    USER_PROMPT = f"context: ```{input}``` \n\n output: "
    messages = [
        {
            "role": "system",
            "content": SYS_PROMPT
        },
        {
            "role": "user", 
            "content": USER_PROMPT
        },
    ]
    prompt = pipe.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    outputs = pipe(
        prompt,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.1,
        top_p=0.95
    )
    ret = outputs[0]["generated_text"]
    # print(ret)
    if (ret[ret.find("[/INST]") + 9: ]).lower().startswith("yes"):
        return True
    else:
        return False
    

In [5]:
data[0]

('/local/sps-local/docs/a5/57/23/d8/a55723d8437655575840c05dbe343a183ace18439654cf6b8e408f62cd8aadcb.json',
 12,
 131,
 [(3, 60, 69), (1, 279, 282)],
 'By using the time (t1) of the first switching crystal as an indicator that the gas has filled the\nchannel we can compare directly the induction time for the following crystals (tn) by defining\nthe normalized switching time 𝑡̂ :\n𝑡̂\n\n𝑡\n\n𝑡\n\n(Eq. 2)\n\nDecreasing the concentration of DCM vapor from 100 to 60%, the distribution of normalized\ninduction time broadens.')

In [ ]:
def normalize_whitespaces(s):
    return " ".join(s.split()).strip()

def process_context(context):
    context_filepath, context_page, context_pos, context_occurences, context_sentence = context    
    chemicals = set()
    roles = set()
    
    for o in context_occurences:
        entity = normalize_whitespaces(context_sentence[o[1]:o[2]])
        if o[0] == 1:
            chemicals.add(entity)
        else:
            roles.add(entity.lower())

    true_relations = []
    false_relations = []
    
    for c in chemicals:        
        for r in roles:
            if answer(context_sentence.replace("\n"," "), c, r):
                true_relations.append((c, r, context_filepath, context_page, context_pos))
            else:
                false_relations.append((c, r, context_filepath, context_page, context_pos))
    return true_relations, false_relations

true_relations, false_relations = [], []

def pickle_relations(true_relations, false_relations):
    with open("/local/sps-local/ner-role-extraction/true_relations.pkl", "wb") as f:
        pickle.dump(true_relations, f)
    with open("/local/sps-local/ner-role-extraction/false_relations.pkl", "wb") as f:
        pickle.dump(false_relations, f)


i = 0
with open("/local/sps-local/ner-role-extraction/llama2-role-validator.log", "w") as log:
    log.write(f"{datetime.now()}: {i} of {len(data)}\n")
    log.flush()
    for context in tqdm(data):
        i += 1
    
        if i%1000 == 0:
            pickle_relations(true_relations, false_relations)
            log.write(f"{datetime.now()}: {i} of {len(data)}\n")
            log.flush()
            
        # print(context)
        tr, fr = process_context(context)
        true_relations.extend(tr)
        false_relations.extend(fr)

true_relations[:5]


  0%|                                                                                                                                                    | 6/115537 [00:01<6:25:59,  4.99it/s]/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 45%|████████████████████████████████████████████████████████████████▏                                                                             | 52255/115537 [4:49:50<5:49:23,  3.02it/s]

In [ ]:
false_relations[:5]

In [ ]:
roles_to_chems = {} # dict of <role, dict<chems, [positional information]>>

for chem, role, context_filepath, context_page, context_pos in true_relations:
    chems = roles_to_chems.get(role, {})
    pos = chems.get(chem, [])
    pos.append((context_filepath, context_page, context_pos))
    chems[chem]=pos
    roles_to_chems[role]=chems

for role in roles_to_chems:
    chems = roles_to_chems.get(role)
    for chem in chems:
        pos = chems[chem] 
        count = len(pos)
        if count > 1:
            print(role, "|", chem, "->", count)        
    